<div class="big_title">LAB 5: Advanced data manipulation</div>

BIO3782: Biologist's Toolkit (Dalhousie University)

----------------------------------------

# Data manipulation using dplyr and tidyr

<span class="important"></span> Bracket subsetting is handy, but it can be cumbersome and difficult to read, especially for complicated operations. Enter **dplyr**. **dplyr** is a package for making tabular data manipulation easier. It pairs nicely with tidyr which enables you to swiftly convert between different data formats for plotting and analysis.

The **tidyverse** package is an "umbrella-package" that installs **tidyr**, **dplyr**, and several other packages useful for data analysis, such as **ggplot2**, **tibble**, etc.

The tidyverse package tries to address 3 common issues that arise when doing data analysis with some of the functions that come with R:

    1. The results from a base R function sometimes depend on the type of data.

    2. Using R expressions in a non standard way, which can be confusing for new learners.
    
    3. Hidden arguments, having default operations that new learners are not aware of.
    
You should already have installed and loaded the **tidyverse** package. If we haven't already done so, we can type install.packages("tidyverse") straight into the console. Then, to load the package type library(tidyverse)



In [ ]:
install.packages("tidyverse")

library(tidyverse)

Installing package into 'C:/Users/flyin/Documents/R/win-library/3.6'
(as 'lib' is unspecified)


________________________________________________________________________________________________

# What are `dplyr` and `tidyr`?

The package **`dplyr`** provides easy tools for the most common data manipulation tasks. It is built to work directly with data frames, with many common tasks optimized by being written in a compiled language (C++). An additional feature is the ability to work directly with data stored in an external database. The benefits of doing this are that the data can be managed natively in a relational database, queries can be conducted on that database, and only the results of the query are returned.

This addresses a common problem with R in that all operations are conducted in-memory and thus the amount of data you can work with is limited by available memory. The database connections essentially remove that limitation in that you can connect to a database of many hundreds of GB, conduct queries on it directly, and pull back into R only what you need for analysis.

The package **`tidyr`** addresses the common problem of wanting to reshape your data for plotting and use by different R functions. Sometimes we want data sets where we have one row per measurement. Sometimes we want a data frame where each measurement type has its own column, and rows are instead more aggregated groups (e.g., a time period, an experimental unit like a plot or a batch number). Moving back and forth between these formats is non-trivial, and **`tidyr`** gives you tools for this and more sophisticated data manipulation.

To learn more about **`dplyr`** and **`tidyr`** after the workshop, you may want to check out this handy data transformation with **`dplyr`** cheatsheet and this one about **`tidyr`**.

First, we'll download the data from figshare using the **`download.file`** function


In [ ]:
download.file(url = "https://ndownloader.figshare.com/files/2292169",
              destfile = "portal_data_joined.csv")

As before, we'll read in our data using the **read_csv()** function from the tidyverse package **readr**.

In [ ]:
surveys <- read_csv("portal_data_joined.csv")

In [ ]:
# inspect data

str(surveys)

In [ ]:
## preview data

view(surveys)

Next, we're going to learn some of the most common **`dplyr`** functions:

* **select()**: subset columns
* **filter()**: subset rows on conditions
* **mutate()**: create new columns by using information from other columns
* **group_by()** and **summarize()**: create summary statistics on grouped data
* **arrange()**: sort results
* **count()**: count discrete values

_________________________________________________________________________________________________________________________

# Selecting columns and filtering rows

To select columns of a data frame, use **select()**. The first argument to this function is the data frame, and the subsequent arguments are the columns to keep.

**select**(data, column names)

In the following code, we have selected the columns "plot_id", "species_id", and "weight". Since the dataset has 34786 rows, we can use the **head()** function to specify the number of rows we would like to see. In this example, we want to see the first four rows only. 

In [ ]:
select(surveys, plot_id, species_id, weight)  %>% 
head(4) # to select the first four rows of each column 

To select all columns except certain ones, put a "-" in front of the variable to exclude it. In the following code, we want to exclude the columns "record_id" and "species_id"

In [ ]:
select(surveys, -record_id, -species_id) %>% 
head(4) # to select the first four rows of each column 

You can also choose rows based on a specific criterion using the **filter()** function. In the following code, we want to select all data from the year 1995 only.

In [ ]:
filter(surveys, year == 1995) %>% 
head(4) # to select the first four rows 

____________________________________________________________________________________________________________________________

# Pipes (%>%)

What if you want to select and filter at the same time? There are three ways to do this: use intermediate steps, nested functions, or pipes.

With intermediate steps, you create a temporary data frame and use that as input to the next function, like this:

In [ ]:
#Creates a new data frame with rows where weight < 5
surveys2 <- filter(surveys, weight < 5)

#Then only selects the columns "species_id", "sex", "weight" , and "record_id" 
surveys_sml <- select(surveys2, record_id, species_id, sex, weight)

This is readable, but can clutter up your workspace with lots of objects that you have to name individually. With multiple steps, that can be hard to keep track of.

You can also nest functions (i.e. one function inside of another), like this:

In [ ]:
surveys_sml <- select(filter(surveys, weight < 5), record_id, species_id, sex, weight)

This is handy, but can be difficult to read if too many functions are nested, as R evaluates the expression from the inside out (in this case, filtering, then selecting).

The last option, pipes, are a recent addition to R. Pipes let you take the output of one function and send it directly to the next, which is useful when you need to do many things to the same dataset. Pipes in R look like **%>%** and are made available via the **`magrittr`** package, installed automatically with **`dplyr`**. 

If you use RStudio, you can type the pipe with **`Ctrl + Shift + M`** if you have a PC or **`Cmd + Shift + M`** if you have a Mac.

In the following example, we want to filter data from the `surveys` dataset, keeping rows where `weight < 5`, then select only the `species_id`, `sex` and `weight` columns. We will use the pipe to send the `surveys` dataset through the `filter()` function followed by the `select()` function. 

In [ ]:
surveys %>%
  filter(weight < 5) %>% 
  select(record_id,species_id, sex, weight)

Since **%>%** takes the object on its left and passes it as the first argument to the function on its right, we don't need to explicitly include the data frame as an argument to the `filter()` and `select()` functions any more.

The **`dplyr`** functions by themselves are somewhat simple, but by combining them into linear workflows with the pipe, we can accomplish more complex manipulations of data frames.

We can also create a new object with this data:

In [ ]:
surveys_sml<-surveys %>%
  filter(weight < 5) %>% 
  select(record_id,species_id, sex, weight)

# Joins

Join functions allow you to connect multiple tables based on common variables(columns) so that you can access info from multiple tables at once. There are (believe it or not) 6 different ways to join tables. The 2nd page of the R data wrangling cheat sheet has a nice summary.

The most useful of these is a **`left join`**, which takes a table (say table 1), and then adds all matching records from table 2.

R will automatically detect which columns two tables have in common IF they are called the same thing. If not, you have to feed the matching columns names into the join function.

## Types of Joins

* **left join**: resulting table will show all records from table 1 (the left hand table), and matching records from table 2 (the right hand table)

* **right join**: result shows all records from table 2, and matching records from table 1

* **inner join**: result shows only 'matched' records - where there is a matching value in both tables

* **full join**: result shows all records from both tables

* **anti join**: removes all records from table 1 that have a match in table 2

* **semi join**: removes all records from table 2 that have a match in table 1

Anti and Semi joins are especially useful when exploring messy data

Let's use the `surveys_sml` dataframe we created previously. Say we have `surveys_sample_dates`, a dataframe of the month, day and year the samplew were collected. We can create a new dataframe with information all the information from both `surveys_sml` and `surveys_sample_dates`


In [ ]:
#
surveys_sample_dates <- select(surveys2, record_id, species_id, month, day, year)

left_join(surveys_sml, surveys_sample_dates, by=c("record_id", "species_id"))



_______________________________________________________________________________________________

# Mutate

We can use **mutate()** to create new columns based on the values in existing columns, for example to do unit conversions, or to find the ratio of values in two columns.

For example, we can create a new column of weight in kg:

In [ ]:
surveys %>%
  mutate(weight_kg = weight / 1000)

We can also create a second column based on values in any of the previous columns in the same call of **mutate()**

In [ ]:
surveys %>%
  mutate(weight_kg = weight / 1000,
         weight_lb = weight_kg * 2.2)

The first few rows of the output are full of NAs, so if we wanted to remove those we could insert a filter() in the chain and use the function `is.na()`. `is.na()` determines whether something is an NA. The `!` symbol negates the result, so if we're asking for every row where weight is not an NA, we use the syntax `!is.na()`.

In [ ]:
surveys %>%
  filter(!is.na(weight)) %>%
  mutate(weight_kg = weight / 1000) %>%
  head()

_______________________________________________________________________________________________

# Split-apply-combine data analysis and the `summarize()` function

<span class="important"></span>Many data analysis tasks can be approached using the split-apply-combine paradigm: 

1. split the data into groups 
1. apply some analysis to each group
1. combine the results. 

dplyr makes this very easy through the use of the **group_by()** function. **group_by()** is often used together with **summarize()**, which collapses each group into a single-row summary of that group. **group_by()** takes as arguments the column names that contain the categorical variables for which you want to calculate the summary statistics. 

In the example below, we compute the mean weight by sex.

In [ ]:
surveys %>% #dataframe
  group_by(sex) %>% #group by criteria
  summarize(mean_weight = mean(weight, na.rm = TRUE)) #function summarize() applies the function mean() to the entirety of the dataframe 

We can also group by mutliple columns. The function **tail()** provides the last 6 (default value) of the summary. 

In [ ]:
surveys %>%
  group_by(sex, species_id) %>%
  summarize(mean_weight = mean(weight, na.rm = TRUE)) %>% 
  tail()

We can see that the sex column contains NA values because some animals had escaped before their sex and body weights could be determined. The resulting mean_weight column does not contain NA but **NaN** (which refers to "Not a Number") because `mean()` was called on a vector of NA values while at the same time setting **na.rm = TRUE**. To avoid this, we can remove the missing values for weight before we attempt to calculate the summary statistics on weight. Because the missing values are removed first, we can omit **na.rm = TRUE** when computing the mean:

In [ ]:
surveys %>%
  filter(!is.na(weight)) %>%
  group_by(sex, species_id) %>%
  summarize(mean_weight = mean(weight))

The output from these calls doesn't run off the screen anymore. If you want to display more data, you can use the print() function at the end of your chain with the argument n specifying the number of rows to display:

In [ ]:
surveys %>%
  filter(!is.na(weight)) %>%
  group_by(sex, species_id) %>%
  summarize(mean_weight = mean(weight)) %>%
  print(n = 15)

You can also summarize multiple variables at the same time (and not necessarily on the same variable). For instance, we could add a column indicating the minimum weight for each species for each sex:

In [ ]:
surveys %>%
  filter(!is.na(weight)) %>% #removes rows where weight = NA
  group_by(sex, species_id) %>% 
  summarize(mean_weight = mean(weight), # computes the mean weight of the dataset
            min_weight = min(weight))  #finds the minimum value of weight

We can also arrange the values in order of minimum weight to put lighter species first using the **arrange()** function. To sort in descending order, we need to add the desc() function. If we want to sort the results by decreasing order of mean weight

In [ ]:
surveys %>%
  filter(!is.na(weight)) %>%
  group_by(sex, species_id) %>%
  summarize(mean_weight = mean(weight),
            min_weight = min(weight)) %>%
  arrange(desc(min_weight))

_______________________________________________________________________________________________

# Counting observations

When working with data, we often want to know the number of observations found for each factor or combination of factors. For this task, dplyr provides **count()**. For example, if we wanted to count the number of rows of data for each sex, we would do:

In [ ]:
surveys %>%
    count(sex) 

The **count()** function is shorthand for something we've already seen: grouping by a variable, and summarizing it by counting the number of observations in that group. In other words, `surveys %>% count()` is equivalent to:

In [ ]:
surveys %>%
    group_by(sex) %>%
    summarise(count = n())

If we wanted to count combination of factors, such as `sex` and `species`, we would specify the first and the second factor as the arguments of **count()**:

In [ ]:
surveys %>%
  count(sex, species) 

We can proceed with **arrange()** to sort the table according to a number of criteria so that we have a better comparison. For instance, we might want to arrange the table above in:

1. an alphabetical order of the levels of the species 
1. in descending order of the count:

In [ ]:
surveys %>%
  count(sex, species) %>%
  arrange(species, desc(n))

From the table above, we may learn that, for instance, there are 75 observations of the albigula species that are not specified for its sex (i.e. NA).

_________________________________________________________________________

# Reshaping your dataframe with **gather()** and **spread()**

Sometimes you may need to alter the layout of tabluar datasets while preserving the values and relationships contained within. In `surveys`, the rows of `surveys` contain the values of variables associated with each record (the unit), values such as the weight or sex of each animal associated with each record. What if instead of comparing records, we wanted to compare the different mean weight of each genus between plots? (Ignoring plot_type for simplicity).

We'd need to create a new table where each row (the unit) is comprised of values of variables associated with each plot. In practical terms this means the values in genus would become the names of column variables and the cells would contain the values of the mean weight observed on each plot.

Having created a new table, it is therefore straightforward to explore the relationship between the weight of different genera within, and between, the plots. The key point here is that we are still following a tidy data structure, but we have **reshaped** the data according to the observations of interest: average genus weight per plot instead of recordings per date.

We can do both these of transformations with two tidyr functions, **spread()** and **gather()**

The opposite transformation would be to transform column names into values of a variable.
**gather()** moves rows into columns (long to wide format) while **spread()** moves columns into rows (wide to long format).

## Spread()

**spread()** takes takes three principal arguments:

1. the data
1. the key column variable whose values will become new column names.
1. the value column variable whose values will fill the new column variables.

Further arguments include fill which, if set, fills in missing values with the value provided.

Let's use **spread()** to transform `surveys` to find the `mean weight` of each genus in each plot over the entire survey period. We use `filter()`, `group_by()`, and `summarise()` to filter our observations and variables of interest, and create a new variable for the mean_weight.

In [ ]:
surveys_gw <- surveys %>%
  filter(!is.na(weight)) %>%
  group_by(plot_id, genus) %>%
  summarize(mean_weight = mean(weight))

str(surveys_gw)

This yields the object **surveys_gw** where the observations for each plot are spread across multiple rows, 196 observations of 3 variables. Using **spread()** to key on genus with values from mean_weight this becomes 24 observations of 11 variables, one row for each plot.

In [ ]:
surveys_spread <- surveys_gw %>%
  spread(key = genus, value = mean_weight)

str(surveys_spread)

THERE IS AN IMAGE HERE

We could now plot comparisons between the weight of genera (one is called a genus, multiple are called genera) in different plots, although we may wish to fill in the missing values first.

In [ ]:
surveys_gw %>%
  spread(genus, mean_weight, fill = 0) %>%
  head()

## Gather()

The opposing situation could occur if we had been provided with data in the form of surveys_spread, where the genus names are column names, but we wish to treat them as values of a genus variable instead.

In this situation we are gathering the column names and turning them into a pair of new variables. One variable represents the column names as values, and the other variable contains the values previously associated with the column names.

**gather()** takes four principal arguments:

1. the data
1. the key column variable we wish to create from column names.
1. the values column variable we wish to create and fill with values associated with the key.
1. the names of the columns we use to fill the key variable (or to drop).

To recreate `surveys_gw` from `surveys_spread` we would create a key called `genus` and value called `mean_weight` and use all columns except `plot_id` for the key variable. Here we exclude plot_id from being gather()ed.

In [ ]:
surveys_gather <- surveys_spread %>%
  gather(key = "genus", value = "mean_weight", -plot_id)

str(surveys_gather)

THERE IS AN IMAGE HERE

Note that now the NA generated are included in the re-gathered format. Spreading and then gathering can be a useful way to balance out a dataset so every replicate has the same composition.

We could also have used a specification for what columns to include. This can be useful if you have a large number of identifying columns, and it's easier to specify what to gather than what to leave alone. And if the columns are directly adjacent, we don't even need to list them all out - just use the **`:`** operator.

In [ ]:
surveys_spread %>%
  gather(key = "genus", value = "mean_weight", Baiomys:Spermophilus) %>%
  head()

_________________________________________________________________________

# Exporting data

Now that you have learned how to use dplyr to extract information from or summarize your raw data, you may want to export these new data sets to share them with your collaborators or for archival.

Similar to the `read_csv()` function used for reading CSV files into R, there is a **write_csv()** function that generates CSV files from data frames.

Before using **write_csv()**, we are going to create a new folder, data, in our working directory that will store this generated dataset. We don't want to write generated datasets in the same directory as our raw data. It's good practice to keep them separate. The data_raw folder should only contain the raw, unaltered data, and should be left alone to make sure we don't delete or modify it. In contrast, our script will generate the contents of the data directory, so even if the files it contains are deleted, we can always re-generate them.

In preparation for our next lesson on plotting, we are going to prepare a cleaned up version of the data set that doesn't include any missing data.

Let's start by removing observations of animals for which weight and hindfoot_length are missing, or the sex has not been determined:

In [ ]:
surveys_complete <- surveys %>%
  filter(!is.na(weight),           # remove missing weight
         !is.na(hindfoot_length),  # remove missing hindfoot_length
         !is.na(sex))                # remove missing sex

Because we are interested in plotting how species abundances have changed through time, we are also going to remove observations for rare species (i.e., that have been observed less than 50 times). We will do this in two steps: first we are going to create a data set that counts how often each species has been observed, and filter out the rare species; then, we will extract only the observations for these more common species:

In [ ]:
## Extract the most common species_id
species_counts <- surveys_complete %>%
    count(species_id) %>% 
    filter(n >= 50)

## Only keep the most common species
surveys_complete <- surveys_complete %>%
  filter(species_id %in% species_counts$species_id)

To make sure that everyone has the same data set, check that surveys_complete has 30463 rows and 13 columns by typing `dim(surveys_complete)`.

Now that our data set is ready, we can save it as a CSV file in our data folder.

In [ ]:
write_csv(surveys_complete, file = "data/surveys_complete.csv")

<span class="console"></span> <br>
----------------------------- <br>
<span class="editor"></span> <br>
----------------------------- <br>
<span class="c2c"></span> <br>
<div class="c2c"></div> <br>
----------------------------- <br>
<span class="run"></span> <br>
<div class="run"></div> <br>
----------------------------- <br>
<span class="runcell"></span> <br>
<div class="runcell"></div> <br>
----------------------------- <br>
<span class="file">Text</span> <br>
<div class="file">Text</div> <br>
----------------------------- <br>
<span class="important"></span> <br>
<div class="important"></div> <br>
----------------------------- <br>
<span class="note"></span> <br>
<div class="note"></div> <br>
----------------------------- <br>
<span class="checkpoint">Text</span> <br>
<div class="checkpoint">Text</div> <br>
-----------------------------
<span class="codeerror">Text</span> <br>
<div class="codeerror">Text</div> <br>
----------------------------- <br>
<span class="lineerror">Text</span> <br>
<div class="lineerror">Text</div> <br>
----------------------------- <br>
<span class="terminal"></span> <br>
<div class="terminal"></div> <br>
----------------------------- <br>
<span class="terminalBlock">Text</span> <br>
<div class="terminalBlock">Text</div> <br>
----------------------------- <br>
<span class="nomenclature">nomenclature</span> <br>
<div class="nomenclature">nomenclature</div> <br>
----------------------------- <br>
<span class="code">code</span> <br>
<div class="code">code</div> <br>
----------------------------- <br>
<span class="rendered_html">rendered_html</span> <br>
<div class="rendered_html">rendered_html</div> <br>
----------------------------- <br>

<br><br><br>
<div class="startTASK"></div>

Write task here

<div class="endTASK"></div>
<br><br><br>



<img src="../media/R_logo.png" width="150px">
<a href="bad_layout.png" target="_blank"><img src="bad_layout.png" width="700px"></a>

<div class="big_title">This is the end of lab</div>

*******************
*******************

Code below is for formatting of this lab. Do not alter!

In [ ]:
cssFile <- '../css/custom.css'
IRdisplay::display_html(readChar(cssFile, file.info(cssFile)$size))

IRdisplay::display_html("<style>body {counter-reset: question_num;}.Q::before {counter-increment: question_num;
    content: 'QUESTION ' counter(question_num) '\\A'; white-space: pre; }</style>")